In [2]:
import flexs

In [1]:
import editdistance
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pprint
import numpy as np
import json

import flexs
from flexs import baselines
import flexs.utils.sequence_utils as s_utils


In [8]:


problem = flexs.landscapes.tf_binding.registry()['SIX6_REF_R1']
pprint.pprint(problem)

landscape = flexs.landscapes.TFBinding(**problem['params'])
alphabet = s_utils.DNAA


bsize=100

{'params': {'landscape_file': '/Users/leofeng/miniconda3/envs/flexsenv/lib/python3.7/site-packages/flexs/landscapes/data/tf_binding/SIX6_REF_R1_8mers.txt'},
 'starts': {1: 'GCTCGAGC',
            2: 'GCGCGCGC',
            3: 'TGCGCGCC',
            4: 'ATATAGCC',
            5: 'GTTTGGTA',
            6: 'ATTATGTT',
            7: 'CAGTTTTT',
            8: 'AAAAATTT',
            9: 'AAAAACGC',
            10: 'GTTGTTTT',
            11: 'TGCTTTTT',
            12: 'AAAGATAG',
            13: 'CCTTCTTT',
            14: 'AAAGAGAG'}}


In [9]:
s_utils.DNAA

'TGCA'

In [10]:

starting_sequence = problem['starts'][3]
starting_sequence

cnn = baselines.models.CNN(len(starting_sequence), alphabet=alphabet,
                         num_filters=32, hidden_size=100, loss='MSE')

random_explorer = baselines.explorers.Random(
    cnn,
    rounds=10,
    mu=1,
    starting_sequence=starting_sequence,
    sequences_batch_size=100,
    model_queries_per_batch=100,
    alphabet=alphabet
)

random_sequences, metadata = random_explorer.run(landscape)
random_sequences

2021-12-02 02:39:47.802102: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


round: 0, top: 0.3138580794128561, time: 0.000269s
round: 1, top: 0.8739228136725353, time: 0.753633s
round: 2, top: 0.9534799818579145, time: 0.130345s
round: 3, top: 0.9534799818579145, time: 0.158309s
round: 4, top: 0.9534799818579145, time: 0.196203s
round: 5, top: 0.9534799818579145, time: 0.225391s
round: 6, top: 0.9534799818579145, time: 0.233839s
round: 7, top: 0.9906300251515278, time: 0.324525s
round: 8, top: 0.9906300251515278, time: 0.326296s
round: 9, top: 0.9906300251515278, time: 0.342116s
round: 10, top: 0.9906300251515278, time: 0.383201s


,sequence,model_score,true_score,round,model_cost,measurement_cost
0,TGCGCGCC,NaN,0.313858,0,0,1
0,TACGCCCC,0.265665,0.567260,1,101,101
1,AGCGCACC,0.220758,0.215478,1,101,101
2,GGAGCGCC,0.297765,0.364378,1,101,101
3,TACACGAG,0.244798,0.611852,1,101,101
...,...,...,...,...,...,...
95,AGCGAATG,0.312130,0.333691,10,1010,1001
96,ACCGCAAC,0.312849,0.337845,10,1010,1001
97,CGTGTGTC,0.582880,0.459758,10,1010,1001
98,TAAGCCTC,0.460585,0.371449,10,1010,1001


In [4]:


cnn = baselines.models.CNN(len(starting_sequence), alphabet=alphabet,
                         num_filters=32, hidden_size=100, loss='MSE')

adalead_explorer = baselines.explorers.Adalead(
    cnn,
    rounds=10,
    starting_sequence=starting_sequence,
    sequences_batch_size=100,
    model_queries_per_batch=2000,
    alphabet=alphabet
)


adalead_sequences, metadata = adalead_explorer.run(landscape)



round: 0, top: 0.28011203300685433, time: 0.000235s
round: 1, top: 0.5190311192130015, time: 14.709052s
round: 2, top: 0.6124017881520295, time: 5.209340s
round: 3, top: 0.7140110778789233, time: 3.763060s
round: 4, top: 0.8732904512231746, time: 3.592170s
round: 5, top: 0.9556763448245889, time: 3.574958s
round: 6, top: 0.9556763448245889, time: 3.848999s
round: 7, top: 0.9556763448245889, time: 3.764802s
round: 8, top: 0.9556763448245889, time: 3.875693s
round: 9, top: 0.9556763448245889, time: 3.830125s
round: 10, top: 0.9556763448245889, time: 3.864334s


In [48]:
cnn = baselines.models.CNN(len(starting_sequence), alphabet=alphabet,
                         num_filters=32, hidden_size=100, loss='MSE')

genetic_explorer = baselines.explorers.GeneticAlgorithm(
    cnn,
    
    population_size=8,
    parent_selection_strategy='wright-fisher', # wright-fisher model decides who gets to 'mate'
    beta=0.01,
    children_proportion=0.2,

    rounds=8,
    starting_sequence=starting_sequence,
    sequences_batch_size=bsize,
    model_queries_per_batch=1000,
    alphabet=alphabet
)

In [49]:
genetic_algo_sequences, metadata = genetic_explorer.run(landscape)

round: 0, top: 0.055, time: 0.000579s
round: 1, top: 0.31, time: 47.479483s


KeyboardInterrupt: 

In [ ]:
cnn = baselines.models.CNN(len(starting_sequence), alphabet=alphabet,
                         num_filters=32, hidden_size=100, loss='MSE')

cmaes_explorer = baselines.explorers.CMAES(
    flexs.LandscapeAsModel(landscape),
    
    population_size=10,
    max_iter=200,
    
    rounds=10,
    starting_sequence=starting_sequence,
    sequences_batch_size=bsize,
    model_queries_per_batch=1000,
    alphabet=alphabet
)

cmaes_sequences, metadata = cmaes_explorer.run(landscape)

In [8]:
cnn = baselines.models.CNN(len(starting_sequence), alphabet=alphabet,
                         num_filters=32, hidden_size=100, loss='MSE')

dynappo_explorer = baselines.explorers.DynaPPO(  # DynaPPO has its own default ensemble model, so don't use CNN
    landscape=landscape,
    env_batch_size=10,
    num_model_rounds=10,
    rounds=8,
    starting_sequence=starting_sequence,
    sequences_batch_size=bsize,
    model_queries_per_batch=1000,
    alphabet=alphabet,
)

dynappo_sequences, metadata = dynappo_explorer.run(landscape)

/Users/leofeng/miniconda3/envs/flexsenv/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)


BoundedTensorSpec(shape=(), dtype=tf.int64, name='action', minimum=array(0), maximum=array(3))
round: 0, top: 0.28011203300685433, time: 0.000227s
round: 1, top: 0.5190311192130015, time: 13.155419s
round: 2, top: 0.6343713912067976, time: 13.785190s
round: 3, top: 0.6975338886827324, time: 23.262424s
round: 4, top: 0.7222496462872485, time: 23.863388s
round: 5, top: 0.7222496462872485, time: 23.563456s
round: 6, top: 0.7332344740044028, time: 25.564283s
round: 7, top: 0.7332344740044028, time: 26.635824s
round: 8, top: 0.7332344740044028, time: 29.754795s
